In [1]:
library(tidyverse)
library(DESeq2)
library(future)
library(furrr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’

In [2]:
dds <- readRDS("/data/rudensky/EYW/git_projects/SIG06/analysis_outs/dds_object_SIG07.rds")

In [8]:
# import feature names (used in function below)
featureNames <- read_csv("/data/rudensky/EYW/git_projects/SIG06/processing_outs/featureNames_SIG06.csv")
featureNames <- select(featureNames, -category)

# sample input can be single character or vector of characters
# control input must be single character, defaults to linker
single_DEG <- function(ligand, control = "linker"){
  # perform IHW DEG analysis
  # don't perform independent filtering because using IHW
  res <- results(dds, contrast = c("treatment",ligand,control),
                 independentFiltering=F,
                 filterFun = ihw)
    
  # perform LFC shrinkage and make tidy
  # pass res to lfcShrink
  resTidy <- lfcShrink(dds, type = "ashr", res=res) %>%
      as_tibble(rownames = "ensembl_ID") %>%
      left_join(featureNames) %>%
      mutate(treatment = ligand) %>%
      arrange(padj)

  return(resTidy)
}

# set options for parallel analysis
plan(multisession, workers=parallelly::availableCores())

Rows: 57186 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): ensembl_ID, gene, category

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [9]:
# select viral treatments groups
viralGroups <- dds$treatment %>% unique()
viralGroups <- viralGroups[grep("recomb|none|linker", viralGroups, invert = T)]
viralGroups

[1] CCL25    IL2      TGFB3    BMP7     MIF      CCL3     IL1F9    IL11    
 [9] IFNB1    LEP      CCL27    BMP4     IL18     IL17F    IL13     TNFSF15 
[17] IL17A    CCL8     IL6      CCL4     IL9      IL27     WNT5A    IL1A    
[25] IL4      CX3CL1   CTF1     GDF7     IL1F8    MSTN     IL1F6    PF4     
[33] TNFSF8   WNT1     TNF      CXCL16   IL1F5    TNFSF10  IL16     IFNK    
[41] CCL9     IL1F10   TGFB1    LIF      TGFB2    CCL21A   GDF11    CXCL9   
[49] CCL12    CXCL12   CCL24    IL21     CCL28    IL33     CCL19    CCL11   
[57] WNT3A    IL17B    INHBA    GDF2     IL23     TSLP     TNFSF9   IL25    
[65] BMP10    TNFSF4   IFNA     CCL22    CCL2     GDF15    CXCL13   CCL26   
[73] IFNG     IL1B     IFNE     CCL7     ADIPOQ   GDF10    IL15     WNT7A   
[81] CCL5     LTA      OSM      CCL17    INHBB    CCL20    IL35     CCL1    
[89] CXCL10   CXCL11   IL12     IL7      IL17C    IL10     TNFSF13B TNFSF13 
[97] TNFSF14  TNFSF12  TNFSF18 
106 Levels: ADIPOQ BMP10 BMP4 BMP7 CCL1 CCL11 CCL12 CCL17 CCL19 CCL2 ... WNT7A

In [10]:
system.time(resFull <- future_map_dfr(c("CXCL9","CXCL10","CXCL11"), ~ single_DEG(.x)))

Timing stopped at: 19.23 0.695 568.6



In [ ]:
resFull <- future_map_dfr(viralGroups, ~ deg_MAST(.x))